In [54]:
import pandas as pd
import matplotlib.pylab as plt
import requests



In [65]:
site = '11161000'
code = '00060'
startDT = '1987-10-01T00:00:00.000-07:00'
endDT = '2023-09-30T00:00:00.000-07:00'

usgs_water_api = ("https://waterservices.usgs.gov/nwis/iv/?format=json&sites=" + site +  "&parameterCd=" + code + 
                  "&siteStatus=all&startDT=" + startDT + "&endDT=" + endDT)
api_response = requests.get(usgs_water_api)

water_data = api_response.json()
data_only = water_data['value']['timeSeries'][0]['values'][0]['value']

streamflow_SC = pd.DataFrame(data_only).sort_values('dateTime')
streamflow_SC.head()

,value,qualifiers,dateTime
0,4.40,"[A, [91]]",1987-10-01T00:15:00.000-07:00
1,4.40,"[A, [91]]",1987-10-01T00:30:00.000-07:00
2,3.90,"[A, [91]]",1987-10-01T00:45:00.000-07:00
3,3.70,"[A, [91]]",1987-10-01T01:00:00.000-07:00
4,3.20,"[A, [91]]",1987-10-01T01:15:00.000-07:00
...,...,...,...
1224946,17.2,[A],2023-09-29T23:00:00.000-07:00
1224947,16.8,[A],2023-09-29T23:15:00.000-07:00
1224948,16.8,[A],2023-09-29T23:30:00.000-07:00
1224949,16.8,[A],2023-09-29T23:45:00.000-07:00


In [66]:
site = '11160500'
code = '00060'
startDT = '1987-10-01T00:00:00.000-07:00'
endDT = '2023-09-30T00:00:00.000-07:00'

usgs_water_api = ("https://waterservices.usgs.gov/nwis/iv/?format=json&sites=" + site +  "&parameterCd=" + code + 
                  "&siteStatus=all&startDT=" + startDT + "&endDT=" + endDT)
api_response = requests.get(usgs_water_api)

water_data = api_response.json()
data_only = water_data['value']['timeSeries'][0]['values'][0]['value']

streamflow_BT = pd.DataFrame(data_only).sort_values('dateTime')
streamflow_BT.head()

,value,qualifiers,dateTime
0,9.40,"[A, [91]]",1987-10-01T00:15:00.000-07:00
1,9.40,"[A, [91]]",1987-10-01T00:30:00.000-07:00
2,9.40,"[A, [91]]",1987-10-01T00:45:00.000-07:00
3,9.40,"[A, [91]]",1987-10-01T01:00:00.000-07:00
4,9.40,"[A, [91]]",1987-10-01T01:15:00.000-07:00
